In [99]:
import pandas as pd

In [100]:
df = pd.read_csv("spotify_millsongdata.csv")

In [101]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [102]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [103]:
df.shape

(57650, 4)

In [104]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [105]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [106]:
df.head(10)

,artist,song,text
0,The Monkees,Tell Me Love,"Oh tell me, love, and tell me truly \r\nDo yo..."
1,Reo Speedwagon,I Do' Wanna Know,You have said as much as you can say \r\nYour...
2,Ella Fitzgerald,Always True To You In My Fashion,If a custom tailored vet \r\nAsks me out for ...
3,Britney Spears,Bombastic Love,I am here to testify \r\nThat you're the only...
4,Hanson,Christmas Time,Everything's different \r\nBut nothing has ch...
5,Radiohead,Optimistic,Flies are buzzing round my head \r\nVultures ...
6,Everclear,New Blue Champion,I don't want to hear you say you're sorry \r\...
7,Black Sabbath,Black Moon,Oh the devil is rising with the moon \r\nHe c...
8,Frank Sinatra,A Wing And A Prayer,And here's one of the nation's top tunes \r\n...
9,Alice In Chains,Queen Of The Rodeo,My daddy was a trucker \r\nLeft home when I w...


In [107]:
df['text'][0]

"Oh tell me, love, and tell me truly  \r\nDo you believe this is forever?  \r\nWell I tell you, love, that I believe you  \r\nOh, when you tell me that you will leave me never  \r\n  \r\nOh tell me, love, that all my words are a waste  \r\nOh tell me, love, are my lips to your taste?  \r\n'Cause I want to please you, I want to please you  \r\nAnd tell me, love, do you know you amaze me?  \r\nYou keep changing for the better with every passing day  \r\nWell, I'll tell you, love, sometimes it kinda scares me  \r\nI've got to keep on getting to you, so you don't get away  \r\n  \r\nOh tell me, love, that all my words are a waste  \r\nAnd tell me, love, are my lips to your taste?  \r\n'Cause I want to please you, I want to please you  \r\nAnd tell me, love, please tell me, love, tell me, love\r\n\r\n"

In [108]:
# df = df.sample(5000)

In [109]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [110]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [111]:
import nltk

nltk.download('punkt')

nltk.data.path.append("path/to/nltk_data")

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [112]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [114]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [115]:
similarity[0]

array([1.        , 0.26110947, 0.00399944, ..., 0.1080527 , 0.02912706,
       0.09351222])

In [116]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [117]:
def recommendation(song_name):
    # Check if the song exists in the DataFrame
    if song_name not in df['song'].values:
        print(f"Song '{song_name}' not found in the DataFrame.")
        return []

    # Get the index of the song
    idx = df[df['song'] == song_name].index[0]

    # Calculate distances and retrieve recommended songs
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    recommended_songs = [df.iloc[m_id[0]].song for m_id in distances[1:21]]

    return recommended_songs

In [118]:
recommendation("Crying Over You")

Song 'Crying Over You' not found in the DataFrame.


[]

In [119]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))